# HANA ML & HANA AI Experiments

## Setup a connection to SAP Datasphere (SAP HANA Cloud database) and use HANA_ML

A notebook to show how to connect to the Datasphere tenant in your SAP Business Data Cloud formation (or standalone) and experiment with its machine learning capabilities.
> Note that this applies to any SAP HANA Cloud system and not just Datasphere, given that SAP HANA Cloud is the database used by SAP Datasphere.

- How to connect to the sql schema exposed by your Datasphere Space
- Using the [hana_ml python library](https://pypi.org/project/hana-ml/) *APL* to build machine learning models directly in Datasphere without copying the data (federated)
See the [hana_ml samples on github](https://github.com/SAP-samples/hana-ml-samples/tree/main) for lots of examples.

### Pre-requisites

First you need to create a db user for your Datasphere Space (you will require *space admin* privileges).  
This db user needs to have `Enable Automated Predictive Library (APL) and Predictive Analysis Library (PAL)` set to TRUE when you create it. This option is only avilable when Dataspheres underlying SAP HANA Cloud db has the script server enabled in *System Configuration -> tenant configuration* (requires a minimum 3 vCPU system sizing).

e.g.

I have a space called `PLAYPEN_JASON_SCOTT` (of type HANA memory & disk).

In the Space settings -> Create a db user. I have specified a user suffix of `MLUSER` so the user schema is `PLAYPEN_JASON_SCOTT#MLUSER`.

Next we will get some sample data to use for the rest of this notebook:
1. Clone repo: https://github.com/SAP-samples/hana-apl-apis-runtimes to your local computer
2. Open the HANA Database Explorer and connect to your exposed schema from your Datasphere Space
3. Right-click over "Catalog" and choose to "Import Catalog Objects" -> select the file: `dataForHANACloud.tar.gz`
4. Ensure to overwrite the schema name to your own exposed schema `PLAYPEN_JASON_SCOTT#MLUSER` as thats the only schema you have full access to.

With our Space exposing an SQL schema and user and some sample data loaded we are ready to build some ML models...

Finally, [Setup a *Python Dev Space* in **SAP Business Application Studio**](https://github.com/SAP-samples/hana-ml-py-codejam/blob/main/exercises/00-setup/setup-bas.md).
> Alternatively you can work on a local machine with vscode (probably the best way is to use docker and a vscode devcontainer for **Anaconda**. This devcontainer comes fully pre-configured for python notebooks and typical data science tasks).

### Install dependencies
Install `hana_ml` and a few other misc. dependencies that can be handy when working in python notebooks.

In [ ]:
!python -m pip install --require-virtualenv -U 'ipykernel' 'ipython' 'nbformat' 'nbconvert'
!python -m pip install --require-virtualenv -U 'hana-ml' 'ipywidgets' 'graphviz'

In [ ]:
!python -m pip list --not-required

In [ ]:
import hana_ml
print (f"SAP HANA Client for Python: {hana_ml.__version__}")

### Setup connection to HANA Cloud DB exposed from Datasphere
This next python cell will ask you to enter the connection details for your exposed Datasphere SQL schema. By doing it this way we don't need to worry about saving passwords into the notebook (*the outputs below will not be captured in git*).  
In a production scenario, however, you should use proper secrets management.

In [ ]:
# The hana_ml data frame is what enables us to work with remote data in the HANA db
# When the collect method is called it copies the data locally into a pandas dataframe.
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext

host_address = input("Enter SAP HANA Cloud host name:")
hdb_user = input("Enter User Name :")
hdb_password = input("Enter Password :")

hana_port = 443
hana_encrypt = True #for HANA Cloud

# Establish connection
conn = dataframe.ConnectionContext(address = host_address,
                                   port = hana_port, 
                                   user = hdb_user, 
                                   password = hdb_password, 
                                   encrypt = hana_encrypt,
                                   sslValidateCertificate = 'false')

In [ ]:
conn.connection.isconnected()

## 📊 What’s a DataFrame?

A **DataFrame** is like a **table in memory** — rows and columns of data you can easily filter, sort, group, or analyze.  

- Each **column** has a name and a data type.  
- Each **row** is an observation or record.  
- It’s fast, flexible, and perfect for working with structured data.  

Think of it as a spreadsheet… but programmable.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35]
})

print(df)

## 🪴 What’s a `hana_ml` DataFrame?

A **`hana_ml` DataFrame** is a **virtual reference to data stored in SAP HANA**, not data in local memory.  

- It **doesn’t copy** the data — it pushes operations down to the HANA database.  
- You can filter, join, aggregate, and apply ML directly **inside HANA**.  
- It looks and feels like a regular DataFrame, but it’s **backed by SQL**.  

Think of it as a “smart SQL view” that behaves like a Python DataFrame.

Let's look at one of the sample data tables inside SAP Datasphere/SAP HANA Cloud.
> Note the `collect()` method on the hana_ml dataframe. This copies the data to the local machine as a pandas dataframe.

In [ ]:
sql_cmd = 'SELECT * FROM AUTO_CLAIMS_FRAUD ORDER BY CLAIM_ID'
hdf_train = dataframe.DataFrame(conn, sql_cmd)
hdf_train.head(6).collect()

Now that we are connected to the exposed sql schema for our Datasphere Space we can use the full power of HANA Cloud and its machine learning libraries. Reasons to use the ML capabilities of HANA include:
- Models may not be available in SAP Databricks (such as AutoML for Classification and Regression)
- No standard BDC Data Products for the dataset
- Models can be exposed as CAP artefacts for use in transactional applications.

# 1. We will now use the APL library in HANA to train a machine learning model for fraud detection
For this fraud detection classification task we will use a gradient boosting binary classifer

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
apl_model = GradientBoostingBinaryClassifier(variable_auto_selection = True)
apl_model.fit(hdf_train, label='IS_FRAUD', key='CLAIM_ID')

## Show an auto-generated report on the training

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(apl_model).build().display()

What variables were excluded from the model?

In [ ]:
df = apl_model.get_debrief_report('ClassificationRegression_VariablesExclusion').collect()
df = df[['Variable', 'Reason For Exclusion']]
df.style.hide(axis='index')

Model performance?

In [ ]:
my_filter = "\"Partition\" = 'Validation' and \"Indicator\" in ('AUC','F1 Score','Cohen''s kappa')"
df = apl_model.get_debrief_report('ClassificationRegression_Performance').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'Value':'{:,.3f}'}
df.style.format(format_dict).hide(axis='index')

Binary target statistics?

In [ ]:
my_filter = "\"Partition\"='Validation'"
df = apl_model.get_debrief_report('BinaryTarget_Statistics').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'% Positive Weight':'{:,.1f}%', '% Negative Weight':'{:,.1f}%', 'Weight':'{:,.0f}'}
df.style.format(format_dict).hide(axis='index')

### ⚠️ Class Imbalance in Fraud Detection

Our validation data shows **only ~10% positive cases (`IS_FRAUD = 1`)** and ~90% negatives.  
This is a **highly imbalanced dataset**, which means:

- A default **0.5 decision threshold** will bias predictions toward the majority class (`No Fraud`).
- Standard accuracy isn’t a good performance measure here.
- Precision, recall, and AUC are more meaningful.

👉 **What to do about it:**
- Use **predicted probabilities** from the model instead of the default label.  
- **Tune the decision threshold** (e.g., using F1 or cost-based optimization).  
- Consider resampling strategies or class weights if needed.  
- Focus on **precision–recall trade-offs**, not just accuracy.


In [ ]:
from hana_ml.model_storage import ModelStorage
model_storage = ModelStorage(connection_context=conn, schema=hdb_user)
apl_model.name = 'My Fraud Model'  
model_storage.save_model(model=apl_model, if_exists='replace')

In [ ]:
model_storage.list_models()

## Load the saved model and make predictions

In [ ]:
from hana_ml.model_storage import ModelStorage
model_storage = ModelStorage(connection_context=conn, schema=hdb_user)
apl_model = model_storage.load_model(name='My Fraud Model')

In [ ]:
info_tables = apl_model.get_model_info()

for i, hdf in enumerate(info_tables, 1):
    print(f"\n=== Model Info Table #{i} ===")
    display(hdf.head(20).collect())   # pull a small sample to pandas for display

In [ ]:
df = apl_model.get_summary().filter("KEY in ('ModelVariableCount', 'ModelSelectedVariableCount', 'ModelRecordCount', 'ModelBuildDate')").collect()
df['KEY'] = df['KEY'].str.replace('Model', '').str.replace('Selected', 'Selected ')
df['KEY'] = df['KEY'].str.replace('Count', ' Count').str.replace('Date', ' Date')
df = df[['KEY','VALUE']]
df.columns = ['Property', 'Value']
df.style.hide(axis='index')

## Make Predictions on New Claims
Load some new claims data

In [ ]:
hdf_new = conn.table('AUTO_CLAIMS_NEW', schema=hdb_user)
hdf_new.head(3).collect()

Predict on the new data (The extra_applyout_settings tells SAP HANA APL to enrich the prediction output with additional columns like probabilities, decision values, and reason codes).

In [ ]:
apl_model.set_params(extra_applyout_settings=
{ 'APL/ApplyExtraMode': 'Advanced Apply Settings', 
  'APL/ApplyPredictedValue': 'true', 
  'APL/ApplyProbability': 'true', 
  'APL/ApplyDecision': 'true', 
  'APL/ApplyReasonCode/TopCount': '3', 
  'APL/ApplyReasonCode/ShowStrengthValue': 'false', 
  'APL/ApplyReasonCode/ShowStrengthIndicator': 'false' }
)
df = apl_model.predict(hdf_new).collect()

In [ ]:
col_dict = {'PREDICTED': 'Target Predicted', 
        'gb_score_IS_FRAUD': 'Score', 
        'gb_proba_IS_FRAUD': 'Probability'}
df.rename(columns=col_dict, inplace=True)
df.columns = [hdr.replace("gb_", "") for hdr in df.columns]
format_dict = {'Probability': '{:,.1%}','Score':'{0:,.2f}'}
df.head(7).style.format(format_dict).hide(axis='index')

# 2. Lets do an APL Regression scenario now...

Load sample census data

In [ ]:
sql_cmd = 'SELECT * FROM CENSUS order by "id"'
hdf_train = dataframe.DataFrame(conn, sql_cmd)
hdf_train.head(6).collect()

## Fit with APL Gradient Boosting

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_regression import GradientBoostingRegressor
apl_model = GradientBoostingRegressor(eval_metric='MAE', variable_auto_selection = True)
apl_model.fit(hdf_train, label='age', key='id')

## Model reports

In [ ]:
df = apl_model.get_debrief_report('ClassificationRegression_VariablesContribution').collect()
df = df.sort_values(by=['Rank'])
df.drop({'Oid','Method','Rank'}, axis=1, inplace=True)
df.drop(df[df.Contribution <=0].index, inplace=True)
format_dict = {'Contribution':'{:,.2%}','Cumulative':'{:,.2%}'}
df.style.format(format_dict).hide(axis='index')

In [ ]:
my_filter = "\"Partition\" = 'Validation' and \"Indicator\" in ('MAPE','RMSE')"
df = apl_model.get_debrief_report('ClassificationRegression_Performance').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'Value':'{:,.3f}'}
df.style.format(format_dict).hide(axis='index')

## Make Predictions

In [ ]:
sql_cmd = 'SELECT * FROM CENSUS LIMIT 100'
hdf_apply = dataframe.DataFrame(conn, sql_cmd)
df = apl_model.predict(hdf_apply).collect()
df.columns = ['id', 'Actual','Prediction']
df.head(8)

# What next...

We have trained classicial machine learning models directly inside the hana database and we never had to replicate the data. We can now iterate opver this process to understand the data better and work toawrd better results ofr your business case.

The models are saved inside hana and we can make use of them in a number of ways... some of which include:

1. Execute the model (inference) on fresh data via SQL (db procedure call). This can then be scheduled via a Task Chain and can provide a nice view over the new data and inference results
1. Execute the model from a CAP application running on BTP (hana_ml can be iused to generate the required CAP artefacts)
1. Create a python app to execute the model with fresh data as a web API and execute it from SAP AI Core / BTP Kyma / BTP Cloud Foundry environments

> Investigate and experiment with the companion package [hana_ai](https://pypi.org/project/hana-ai/) where you can use plain english text to generate your hana_ml code (this uses GenAI models via SAP AI Core).